```{contents}
```
## Graph Viewer

The **Graph Viewer** in LangGraph is a visualization and inspection facility that renders the **structure and execution behavior of a LangGraph workflow** as a directed graph.
It is essential for **design verification, debugging, optimization, documentation, and production observability**.

---

### **1. Purpose and Intuition**

A LangGraph program is a **state machine with control flow, loops, and conditional routing**.
Without visualization, the behavior becomes opaque as the system grows.

The Graph Viewer provides:

| Capability               | Why It Matters                     |
| ------------------------ | ---------------------------------- |
| Structural visualization | Understand workflow topology       |
| Execution tracing        | See actual execution path          |
| Loop inspection          | Verify termination behavior        |
| Debugging                | Find dead-ends & incorrect routing |
| Documentation            | Communicate system design          |
| Production analysis      | Detect bottlenecks & failures      |

---

### **2. What the Graph Viewer Shows**

| Element          | Representation        |
| ---------------- | --------------------- |
| Node             | Box                   |
| Edge             | Arrow                 |
| Conditional Edge | Labeled arrow         |
| Entry Node       | Highlighted           |
| Terminal Node    | Marked END            |
| Cycles           | Backward edges        |
| Subgraphs        | Nested clusters       |
| Parallelism      | Split & join branches |

---

### **3. Core Visualization APIs**

LangGraph exposes the viewer through the compiled graph object.

```python
graph.get_graph()
```

### **Static Image Rendering**

```python
from IPython.display import Image, display
display(Image(graph.get_graph().draw_png()))
```

Generates a **Graphviz PNG**.

### **Interactive Diagram (Mermaid)**

```python
display(graph.get_graph().draw_mermaid())
```

Renders an **interactive HTML diagram** (no native dependencies).

---

### **4. Example: Visualizing a Cyclic Graph**

```python
from langgraph.graph import StateGraph, END
from typing import TypedDict

class State(TypedDict):
    count: int

def inc(s): return {"count": s["count"] + 1}
def check(s): return s

builder = StateGraph(State)
builder.add_node("inc", inc)
builder.add_node("check", check)

builder.set_entry_point("inc")
builder.add_edge("inc", "check")
builder.add_conditional_edges("check", lambda s: "inc" if s["count"] < 3 else END, {
    "inc": "inc",
    END: END
})

graph = builder.compile()
display(Image(graph.get_graph().draw_png()))
```

The viewer will show:

```
inc → check → inc → check → ... → END
```

---

### **5. Execution Trace Visualization (LangSmith)**

Graph Viewer shows **structure**.
For **runtime behavior**, enable tracing:

```python
graph.invoke(input, config={"callbacks": [LangChainTracer()]})
```

In LangSmith you see:

* Node execution order
* State changes
* Timing & latency
* Token & cost usage
* Errors & retries

---

### **6. Production Use of Graph Viewer**

| Phase        | Usage                  |
| ------------ | ---------------------- |
| Design       | Validate architecture  |
| Development  | Debug routing & loops  |
| Testing      | Verify all branches    |
| Deployment   | Document system        |
| Operations   | Monitor live execution |
| Optimization | Identify slow paths    |

---

### **7. Viewer vs Observability Stack**

| Tool         | Role                       |
| ------------ | -------------------------- |
| Graph Viewer | Static & logical structure |
| LangSmith    | Execution traces           |
| Logs         | Detailed events            |
| Metrics      | Performance health         |
| Alerts       | Failure detection          |

---

### **8. Best Practices**

* Visualize after each major refactor
* Store graph images with version control
* Use Mermaid for CI documentation
* Combine with LangSmith for runtime insight
* Limit node fan-out for readability

---

### **9. Mental Model**

> **Graph Viewer = X-ray of your agent system**

It exposes the **true control structure** of your LLM system and is indispensable for building **safe, reliable, scalable LangGraph applications**.


### Demonstration

In [1]:
# One-cell complete demonstration of LangGraph Graph Viewer

from typing import TypedDict
from langgraph.graph import StateGraph, END
from IPython.display import Image, display

# 1️⃣ Define shared state
class State(TypedDict):
    count: int

# 2️⃣ Define nodes
def increment(state: State):
    return {"count": state["count"] + 1}

def check(state: State):
    return state

# 3️⃣ Build graph
builder = StateGraph(State)
builder.add_node("increment", increment)
builder.add_node("check", check)

builder.set_entry_point("increment")
builder.add_edge("increment", "check")

# 4️⃣ Create cycle using conditional edge
builder.add_conditional_edges(
    "check",
    lambda s: "increment" if s["count"] < 3 else END,
    {"increment": "increment", END: END}
)

# 5️⃣ Compile graph
graph = builder.compile()

# 6️⃣ Visualize graph
display(Image(graph.get_graph().draw_png()))

# 7️⃣ Execute graph
result = graph.invoke({"count": 0})
print("Final State:", result)


ImportError: Install pygraphviz to draw graphs: `pip install pygraphviz`.

In [2]:
!pip3.12 install pygraphviz

  Using cached pygraphviz-1.14.tar.gz (106 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build pygraphviz


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [88 lines of output]
  C:\Users\sangouda\AppData\Local\Temp\pip-build-env-b47xjp9z\overlay\Lib\site-packages\setuptools\config\_apply_pyprojecttoml.py:82: SetuptoolsDeprecationWarning: `project.license` as a TOML table is deprecated
  !!
  
          ********************************************************************************
          Please use a simple string containing a SPDX expression for `project.license`. You can also use `project.license-files`. (Both options available on setuptools>=77.0.0).
  
          By 2026-Feb-18, you need to update your project and remove deprecated calls
          or your builds will no longer be supported.
  
          See https://packaging.python.org/en/latest/guides/writing-pyproject-toml/#license for details.
          ********************************************************************************
  
  !!
    corresp(dist, value, root_dir)
  C:\Users\sangouda\AppData\Local\Temp\pip-